# Data locality

 - Author: Elwin van 't Wout
 - Affiliation: Pontificia Universidad Católica de Chile
 - Date: 18-8-2023

Test the efficiency of Python with different memory access.

In [1]:
import numpy as np

## Loop strides

Test the influence of the stride of the loop on the efficiency. First, create a large NumPy array with random numbers. Then, perform two different sums, with the same number of operators.

A loop with ```range(0,N,1)``` has elements 0, 1, 2, 3, ..., N-1.

A loop with ```range(0,N*s,s)``` has elements 0, s, 2s, 3s, ..., (N-1)s.


In [2]:
N = 1000000
stride = 187

In [3]:
%%time
a = np.random.rand(N)

CPU times: user 8.67 ms, sys: 9.03 ms, total: 17.7 ms
Wall time: 14.9 ms


In [4]:
%%timeit

sum1 = 0.0
for n in range(0,N,1):
    sum1 += a[n]


109 ms ± 6.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%%time
b = np.random.rand(N*stride)

CPU times: user 1.09 s, sys: 1.04 s, total: 2.13 s
Wall time: 2.13 s


In [6]:
%%timeit

sum2 = 0.0
for n in range(0,N*stride,stride):
    sum2 += b[n]


149 ms ± 4.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


This experiment shows two algorithms, each with the same number of the same operations: $N$ summations. However, the timing is different. This can only be due to different memory access.

The efficiency of a NumPy array is different than for a Python list because both store the data in diferent formats.

In [7]:
%%time
c = [np.random.rand() for n in range(N*stride)]

CPU times: user 49.8 s, sys: 2.87 s, total: 52.7 s
Wall time: 52.8 s


In [8]:
%%timeit

sum3 = 0.0
for n in range(0,N*stride,stride):
    sum3 += c[n]


158 ms ± 9.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In general, it is almost always more efficient to use Numpy functionality.

In [9]:
%%timeit

sum4 = np.sum(a)


337 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%%timeit

sum5 = np.sum(b[::stride])


9.11 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit

sum6 = np.sum(c[::stride])


153 ms ± 8.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Summing the elements of a multi-dimensional arrays

The elements of a multidimensional arrays are stored in memory as a one-dimensional ordering. Hence, the order of accessing the elements has an impact on the timing. Let us create a 3-dimensional tensor and sum all elements. Again, the different implementations all require exactly the same number of the same operators ($N^3$ summations) but the memory access is different.

In [12]:
N = 250

a = np.random.rand(N,N,N)
b = np.random.rand(N,N,N)
c = np.random.rand(N,N,N)

print("Created a random array of shape",a.shape)

Created a random array of shape (250, 250, 250)


In [13]:
%%timeit

sum1 = 0.0
for i in range(N):
    for j in range(N):
        for k in range(N):
            sum1 += a[i,j,k]


2.39 s ± 292 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit

sum2 = 0.0
for k in range(N):
    for j in range(N):
        for i in range(N):
            sum2 += b[i,j,k]


3.44 s ± 322 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Instead of using a loop, it is more efficient to use NumPy functions that are based on optimised algorithms and implementations.

In [15]:
%%timeit

sum3 = np.sum(c)


8.93 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Python broadcasting

NumPy has optimised implementations for many algorithms. For example, summing a constant value to all elements in an array can be done with ```a+=1```. Even though the variable ```a``` has size ```(n,)``` and ```1``` is a scalar, the algorithm works. NumPy uses *broadcasting* which means that (if possible) the summation is performed for all elements.

In [16]:
N = 1000000

In [17]:
%%timeit

a = np.zeros(N)
for n in range(N):
    a[n] += 1


153 ms ± 2.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit

b = np.zeros(N)
b += 1


837 µs ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Matrix-matrix multiplication

Perform a matrix-matrix multiplication with different types of memory access.

In [19]:
n = 500

A = np.random.rand(n,n)
B = np.random.rand(n,n)

In [20]:
%%timeit

C1 = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        C1[i,j] = np.dot(A[i,:],B[:,j])


355 ms ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit

C2 = np.zeros((n,n))
for j in range(n):
    for k in range(n):
        C2[:,j] += A[:,k]*B[k,j]


727 ms ± 28.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit

C3 = np.zeros((n,n))
for i in range(n):
    for k in range(n):
        C3[i,:] += A[i,k]*B[k,:]


539 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit

C4 = A @ B


3.28 ms ± 668 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Consider a larger matrix size.

In [24]:
import time

n = 2000
A = np.random.rand(n,n)
B = np.random.rand(n,n)


C1 = np.zeros((n,n))
time_start = time.time()
for i in range(n):
    for j in range(n):
        C1[i,j] = np.dot(A[i,:],B[:,j])
time_end = time.time()
print("Finished the loops with order (i,j,*) in",time_end-time_start,"seconds.")

C2 = np.zeros((n,n))
time_start = time.time()
for j in range(n):
    for k in range(n):
        C2[:,j] += A[:,k]*B[k,j]
time_end = time.time()
print("Finished the loops with order (j,k,*) in",time_end-time_start,"seconds.")

C3 = np.zeros((n,n))
time_start = time.time()
for i in range(n):
    for k in range(n):
        C3[i,:] += A[i,k]*B[k,:]
time_end = time.time()
print("Finished the loops with order (i,k,*) in",time_end-time_start,"seconds.")

C4 = np.zeros((n,n))
time_start = time.time()
C4 = A @ B
time_end = time.time()
print("Finished the Numpy algorithm in",time_end-time_start,"seconds.")


Finished the loops with order (i,j,*) in 15.621845722198486 seconds.
Finished the loops with order (j,k,*) in 58.30747675895691 seconds.
Finished the loops with order (i,k,*) in 15.23149061203003 seconds.
Finished the Numpy algorithm in 0.11636805534362793 seconds.


## Apending data

Appending data to a Python list is efficient, but Numpy appends arrays by copying the data.

In [25]:
n = 100000

In [26]:
%%timeit

my_list = []

for i in range(n):
    my_list.append(i)

3.57 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
%%timeit

my_array1 = np.array([], dtype=int)

for i in range(n):
    my_array1 = np.append(my_array1, i)

1.66 s ± 22.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit

my_array2 = np.empty(n, dtype=int)

for i in range(n):
    my_array2[i] = i

5.76 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
